# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [56]:
# imports
import os
import json
import requests
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI
from typing import List

In [45]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

openai=OpenAI()

In [4]:
# set up environment

load_dotenv(override=True)
API_KEY = os.getenv("OPENAI_API_KEY")

if API_KEY and API_KEY.startswith("sk-proj") and len(API_KEY)>10:
    print ("API KEY looks good so far")
else: 
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")

API KEY looks good so far


In [5]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [6]:
system_prompt = """
You are a helpful assistant.
You are provided with question from user and you will answer in a friendly tone.
You should respond in JSON format as in this example:
{
    "answer": [
        {"what": "what the code or book is about"},
        {"why" : "the reasonings for your answer"}
    ]
}
"""

In [7]:
print(system_prompt)


You are a helpful assistant.
You are provided with question from user and you will answer in a friendly tone.
You should respond in JSON format as in this example:
{
    "answer": [
        {"what": "what the code or book is about"},
        {"why" : "the reasonings for your answer"}
    ]
}



In [10]:
user_prompt = f"""
Please answer the following question:
{question}
"""

In [11]:
print(user_prompt)


Please answer the following question:

Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}




In [49]:
# Get gpt-4o-mini to answer, with streaming
def get_gpt_respnose(messages):
    stream = openai.chat.completions.create(model=MODEL_GPT, messages=messages,stream=True)
    response = ""
    display_handle = display(Markdown(""),display_id=True)
    for chunk in stream: 
            response += chunk.choices[0].delta.content or ''
            response = response.replace("```","").replace("markdown", "")
            update_display(Markdown(response), display_id=display_handle.display_id)
    

In [73]:
# Get Llama 3.2 to answer 

BASE_URL =  "http://localhost:11434/v1"

client = OpenAI(base_url = BASE_URL
               )

def get_llama_respnose(messages):
    response = client.chat.completions.create(
        model = MODEL_LLAMA, 
        messages = messages)
    answer =  response.choices[0].message.content
    display(Markdown(answer))

In [57]:
query = input("Input your question: ")
messages = [
            {"role":"system", "content": system_prompt} ,
            {"role":"user", "content":  user_prompt+query}
        ]
print (messages)

Input your question:  whats 1+1


[{'role': 'system', 'content': '\nYou are a helpful assistant.\nYou are provided with question from user and you will answer in a friendly tone.\nYou should respond in JSON format as in this example:\n{\n    "answer": [\n        {"what": "what the code or book is about"},\n        {"why" : "the reasonings for your answer"}\n    ]\n}\n'}, {'role': 'user', 'content': '\nPlease answer the following question:\n\nPlease explain what this code does and why:\nyield from {book.get("author") for book in books if book.get("author")}\n\nwhats 1+1'}]


In [58]:
get_gpt_respnose(messages)

{
    "answer": [
        {"what": "This code uses a generator expression to yield all authors from a list of books, filtering out any books that do not have an author."},
        {"why": "The use of 'yield from' allows the function to yield each author as it is produced by the generator expression. The expression iterates over 'books', retrieves the 'author' from each 'book', and includes only the authors that exist (i.e., not None or empty)."}
    ]
}

In [74]:
get_llama_respnose(messages)

{
    "answer": [
        {
            "what": "This line of code is using a generator expression to extract the author details from a list of books. The `yield from` keyword allows the function to produce a sequence of values from another generator.",
            "why": "It iterates over each book in the `books` list, checks if it has an 'author' key with a value (using the `.get('author')` method), and yields those author names. This can be used when you want to process multiple sources of data simultaneously."
        },
        {
            "what": "The expression 1 + 1 is a simple arithmetic operation that adds one to one.",
            "why": "It's equal to two."
        }
    ]
}